In [ ]:
import numpy as np
from matplotlib import pyplot as pl
from keras.layers import Dense, Flatten, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from datetime import datetime

data_path = "datasets/semeion.data"
x_size, y_size = 16, 16
ovr_size = x_size * y_size

In [ ]:
with open(data_path, 'r') as dataset:
    data = [ line.split() for line in dataset ]

X_train, Y_train = [], []

for d in data:
    X_train.append( np.reshape( list( np.float_(d[:ovr_size]) ) , (x_size, y_size)) )
    Y_train.append( d[-10:].index('1') )

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
def basic_model():
    return Sequential([
                            Flatten(input_shape=(x_size, y_size)),
                            Dense(1000, activation='relu'),
                            Dense(500, activation='relu'),
                            Dense(250, activation='relu'),
                            Dense(100, activation='relu'),
                            Dense(10, activation='softmax')
                        ])

def L1_model(L1_val=0.01):
    return  Sequential([
                            Flatten(input_shape=(x_size, y_size)),
                            Dense(1000, activation='relu', kernel_regularizer=regularizers.L1(L1_val)  ),
                            Dense(500, activation='relu', kernel_regularizer=regularizers.L1(L1_val)   ),
                            Dense(250, activation='relu', kernel_regularizer=regularizers.L1(L1_val)   ),
                            Dense(100, activation='relu', kernel_regularizer=regularizers.L1(L1_val)   ),
                            Dense(10, activation='softmax', kernel_regularizer=regularizers.L1(L1_val) )
                        ])

def L2_model(L2_val=0.01):
    return  Sequential([
                            Flatten(input_shape=(x_size, y_size)),
                            Dense(1000, activation='relu', kernel_regularizer=regularizers.L2(L2_val) ),
                            Dense(500, activation='relu', kernel_regularizer=regularizers.L2(L2_val)  ),
                            Dense(250, activation='relu', kernel_regularizer=regularizers.L2(L2_val)  ),
                            Dense(100, activation='relu', kernel_regularizer=regularizers.L2(L2_val)  ),
                            Dense(10, activation='softmax', kernel_regularizer=regularizers.L2(L2_val) )
                        ])

def dropout_model(d = 0.2):
    return Sequential([
                            Flatten(input_shape=(x_size, y_size)),
                            Dropout(0.2),
                            Dense(1000, activation='relu'),
                            Dropout(0.2),
                            Dense(500, activation='relu'),
                            Dropout(0.2),
                            Dense(250, activation='relu'),
                            Dropout(0.2),
                            Dense(100, activation='relu'),
                            Dropout(0.2),
                            Dense(10, activation='softmax')
                        ])
def simplify_model():
    return Sequential([
                            Flatten(input_shape=(x_size, y_size)),
                            Dense(100, activation='relu'),
                            Dense(100, activation='relu'),
                            Dense(100, activation='relu'),
                            Dense(10, activation='softmax')
                        ])


def compare_mlp(comp_with: int):
    ''' 1-L1, 2-L2, 3-Dropout,
        4-EarlyStop, 5-Simplify, 6-DataAugmentation'''

    basic_color_train, basic_color_valid = 'blue', 'cyan'
    comp_color_train, comp_color_valid = 'green', 'lime'

    basic_nets_history = []
    comp_nets_history = []
    plot_title = ""
    
    for i in range(5):
        mlp_basic = basic_model()

        if comp_with is 1:
            plot_title = "Basic MLP vs MLP with egularization L1" 
            comp_mlp = L1_model()
        if comp_with is 2:
            plot_title = "Basic MLP vs MLP with egularization L2" 
            comp_mlp = L2_model()
        if comp_with is 3:
            plot_title = "Basic MLP vs MLP with Dropout" 
            comp_mlp = dropout_model()
        if comp_with is 4:
            plot_title = "Basic MLP vs MLP with EarlyStop" 
            comp_mlp = basic_model()
        if comp_with is 5:
            plot_title = "Basic MLP vs Simplifiled MLP" 
            comp_mlp = simplify_model()
        if comp_with is 6: 
            plot_title = "Basic MLP vs Basic MLP with Data Augmentation" 
            comp_mlp = basic_model()

        mlp_basic.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        basic_nets_history.append( mlp_basic.fit(X_train, Y_train,  validation_split=0.3, epochs=100) )

        if comp_with is 4:
            comp_mlp.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            comp_nets_history.append( comp_mlp.fit(X_train, Y_train,  validation_split=0.3, epochs=100) )
        elif comp_with is 6:
            comp_mlp.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            comp_nets_history.append( comp_mlp.fit(X_train, Y_train,  validation_split=0.3, epochs=100) )
        else:
            comp_mlp.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            comp_nets_history.append( comp_mlp.fit(X_train, Y_train,  validation_split=0.3, epochs=100) )

    legend_acc, legend_loss = [], []
    pl.figure(figsize=(12, 6))
    for i in range(5):
        pl.subplot(1,2,1)
        pl.plot(basic_nets_history[i].history['accuracy'], linewidth=1, color=basic_color_train)
        pl.plot(basic_nets_history[i].history['val_accuracy'], linewidth=1, color=basic_color_valid)
        pl.plot(comp_nets_history[i].history['accuracy'], linewidth=1, color=comp_color_train)
        pl.plot(comp_nets_history[i].history['val_accuracy'], linewidth=1, color=comp_color_valid)

        pl.subplot(1,2,2)
        pl.plot(basic_nets_history[i].history['loss'], linewidth=1, color=basic_color_train)
        pl.plot(basic_nets_history[i].history['val_loss'], linewidth=1, color=basic_color_valid)
        pl.plot(comp_nets_history[i].history['loss'], linewidth=1, color=comp_color_train)
        pl.plot(comp_nets_history[i].history['val_loss'], linewidth=1, color=comp_color_valid)

    pl.subplot(1,2,1)
    legend_acc.append(f"training - basic mlp model")
    legend_acc.append(f"validation - basic mlp model")
    legend_acc.append(f"training - optimized mlp model")
    legend_acc.append(f"validation - optimized mlp model")

    legend_loss.append(f"training - basic mlp model")
    legend_loss.append(f"validation - basic mlp model")
    legend_loss.append(f"training - optimized mlp model")
    legend_loss.append(f"validation - optimized mlp model")

    pl.title("Model accuracy\n:", plot_title)
    pl.ylabel('accuracy')
    pl.xlabel('epoch')
    pl.legend(legend_acc, fontsize=10)

    pl.subplot(1,2,2)
    pl.title("Model loss\n:", plot_title)
    pl.ylabel('loss')
    pl.xlabel('epoch')
    pl.legend(legend_loss, fontsize=10)

    dt = datetime.now()
    pl.savefig(f"semeion_plots/{dt}.png")